# Rf 6 0 6_Nllerrorhandling
Likelihood and minimization: understanding and customizing error handling in likelihood evaluations



**Author:** 07/2008 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:59 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooArgusBG.h"
#include "RooNLLVar.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "RooPlot.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

Create model and dataset
 ----------------------------------------------

Observable

In [3]:
RooRealVar m("m", "m", 5.20, 5.30);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Parameters

In [4]:
RooRealVar m0("m0", "m0", 5.291, 5.20, 5.30);
RooRealVar k("k", "k", -30, -50, -10);

Pdf

In [5]:
RooArgusBG argus("argus", "argus", m, m0, k);

Sample 1000 events in m from argus

In [6]:
RooDataSet *data = argus.generate(m, 1000);

Plot model and data
 --------------------------------------

In [7]:
RooPlot *frame1 = m.frame(Bins(40), Title("Argus model and data"));
data->plotOn(frame1);
argus.plotOn(frame1);

Fit model to data
 ---------------------------------

The argus background shape has a sharp kinematic cutoff at m=m0
 and is prone to evaluation errors if the cutoff parameter m0
 is floated: when the pdf cutoff value is lower than that in data
 events with m>m0 will have zero probability

Perform unbinned ml fit. print detailed error messages for up to
 10 events per likelihood evaluation. The default error handling strategy
 is to return a very high value of the likelihood to MINUIT if errors occur,
 which will force MINUIT to retreat from the problematic area

In [8]:
argus.fitTo(*data, PrintEvalErrors(10));

[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 k           -3.00000e+01  4.00000e+00   -5.00000e+01 -1.00000e+01
     2 m0           5.29100e+00  4.50000e-03    5.20000e+00  5.30000e+00
 **********
 **    3 **SET ERR         0.5
 **********
 **********
 **    4 **SET PRINT           1
 **********
 **********
 **    5 **SET STR           1
 **********
 NOW USING STRATEGY  1: TRY TO BALANCE SPEED AGAINST RELIABILITY
 **********
 **    6 **MIGRAD        1000           1
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
 START MIGRAD MINIMIZATION.  STRATEGY  1.  CONVERGENCE WHEN EDM .LT. 1.00e-03
 FCN=-2417.48 FROM MIGRAD    STATUS=INITIATE        6 CALLS           7 TOTAL
                     EDM= unknown      STRATEGY= 1      NO ERROR MATRIX

Perform another fit. in this configuration only the number of errors per
 likelihood evaluation is shown, if it is greater than zero. The
 EvalErrorWall(kFALSE) arguments disables the default error handling strategy
 and will cause the actual (problematic) value of the likelihood to be passed
 to MINUIT.

 NB: Use of this option is NOT recommended as default strategy as broken -log(L) values
 can often be lower than 'good' ones because offending events are removed.
 This may effectively create a false minimum in problem areas. This is clearly
 illustrated in the second plot

In [9]:
m0.setError(0.1);
argus.fitTo(*data, PrintEvalErrors(0), EvalErrorWall(kFALSE));

[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
 **********
 **   10 **SET PRINT           1
 **********
 **********
 **   11 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 k           -3.53645e+01  3.51934e+00   -5.00000e+01 -1.00000e+01
     2 m0           5.29040e+00  1.00000e-01    5.20000e+00  5.30000e+00
 MINUIT WARNING IN PARAMETR
 ============== VARIABLE2 BROUGHT BACK INSIDE LIMITS.
 MINUIT WARNING IN PARAMETR
 ============== VARIABLE2 BROUGHT BACK INSIDE LIMITS.
 **********
 **   12 **SET ERR         0.5
 **********
 **********
 **   13 **SET PRINT           1
 **********
 **********
 **   14 **SET STR           1
 **********
 NOW USING STRATEGY  1: TRY TO BALANCE SPEED AGAINST RELIABILITY
 **********
 **   15 **MIGRAD        1000           1
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
 START MIGRAD MINIMIZATION.  STRATEGY  1.  CONVERGENCE WHEN EDM

Plot likelihood as function of m0
 ------------------------------------------------------------------

Construct likelihood function of model and data

In [10]:
RooNLLVar nll("nll", "nll", argus, *data);

Plot likelihood in m0 in range that includes problematic values
 In this configuration no messages are printed for likelihood evaluation errors,
 but if an likelihood value evaluates with error, the corresponding value
 on the curve will be set to the value given in EvalErrorValue().

In [11]:
RooPlot *frame2 = m0.frame(Range(5.288, 5.293), Title("-log(L) scan vs m0, problematic regions masked"));
nll.plotOn(frame2, PrintEvalErrors(-1), ShiftToZero(), EvalErrorValue(nll.getVal() + 10), LineColor(kRed));
frame2->SetMaximum(15);
frame2->SetMinimum(0);

TCanvas *c = new TCanvas("rf606_nllerrorhandling", "rf606_nllerrorhandling", 1200, 400);
c->Divide(2);
c->cd(1);
gPad->SetLeftMargin(0.15);
frame1->GetYaxis()->SetTitleOffset(1.4);
frame1->Draw();
c->cd(2);
gPad->SetLeftMargin(0.15);
frame2->GetYaxis()->SetTitleOffset(1.4);
frame2->Draw();

Draw all canvases 

In [12]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()